### For developers wanting to extend the functionality of cyclops.query API for existing datasets or add new datasets, this in-depth guide can be a resource. The walkthrough explanations are using the MIMIC-IV dataset.

### This notebook dives into the details of how to use SQLAlchemy, which is the main python library used to develop the cyclops.query API. 

## SQLAlchemy

SQLAlchemy (https://www.sqlalchemy.org/) is a Python SQL toolkit which implements an Object Relational Mapper (ORM).

An ORM allows users to stay away writing queries as raw SQL strings, which become highly unmanageable as more complex queries are required.

In this tutorial, we'll introduce some basic functionalities provided by SQLAlchemy's ORM, as well as the functionality we've developed for simplifying and extending the use of this ORM.

Note: cyclops.query API is built for querying from a database, not on modifying it, e.g., inserting or deleting rows/tables.

## Query objects

There are a few objects representing queries/relations to understand:
 - `cyclops.query.utils.DBTable` (DBTable)
  - A class defined in this repository used by `cyclops.orm.Database` to organize tables as attributes
 - `sqlalchemy.sql.schema.Table` (Table)
  - The SQLAlchemy table object
  - The DBTable object has attribute `data` which accesses the corresponding Table object
 - `sqlalchemy.sql.selectable.Select` (Select)
  - The SQLAlchemy object returned by function `sqlalchemy.select`
  - Executing queries requires them to be Select objects
 - `sqlalchemy.sql.selectable.Subquery` (Subquery)
  - The SQLAlchemy object returned by method `.subquery()`, e.g., on a `sqlalchemy.select` object
  - This is necessary when wanting to chain queries together

## Query Interface

We wish to provide simple, and easy to use functions for querying a database such as MIMIC-IV, while leveraging the abstraction and flexibility provided by SQLAlchemy's ORM. 

In order to achieve this, the `cyclops.orm.Database` object which encapsulates the ORM, is hidden away from the user. Only the query and resulting data is made available using `cyclops.query.interface.QueryInterface`.

Hence, queries such patients, diagnoses, labs, etc. are encapsulated within this interface, and the user would execute the query using a simple `run()` method which belongs to the interface. The query is then executed and the resulting data as a `pandas.DataFrame` is stored in the interface's `data` attribute.

## API methods

Typically for health datasets collected from hospitals, such as MIMIC-IV, there are multiple tables where different sources of information are available. Usually the following sources are commonly available:

* Patients
* Diagnoses
* Events or Measurements
* Notes

Events can further be categorised into labs, vital measurements, interventions. A researcher or user is interested in developing risk-predictive models that predict outcomes of interest for a patient, which use patient statics and events/measurements as inputs.

Hence, as part of any dataset's query API, we can define functions that allow users to query the above information, and apply different filters to get subsets of interest. A researcher might be interested in patients belonging to a specific cohort depending on an outcome of interest, for example patients that have heart conditions or have had heart transplants. 

```python
    def patients():
        pass
    
    def diagnoses():
        pass
        

## Attribute access

Attributes, or columns, are accessed by name. How to access an column depends on the object.

In [ ]:
# DBTable
col = db.mimic_hosp.d_icd_diagnoses.icd_code
type(col)

There is no convenient attribute access for a `sqlalchemy.sql.schema.Table` object. This was largely the motivation for creating the `cyclops.query_utils.DBTable` object.

There is no direct attribute access for a `sqlalchemy.sql.selectable.Select` object. Instead, we must first convert it to a subquery.

In [ ]:
# Subquery
subquery = select(db.mimic_hosp.d_icd_diagnoses.data).subquery()
type(subquery.c.icd_code)

Note that difference in attribute access between a DBTable and a subquery, where in a subquery, we must use access `.c` before accessing column.

# Helper functions for querying

SQLAlchemy does not have implementations for many desired functionalities.

Here, we introduce `cyclops.query.utils` (imported as `query_utils`), which provides general-purpose functions we'll use throughout the remainder of the notebook to make building effective queries quick and simple. These helper functions can be used to add/improve the higher level API functions in the dataset API modules such as `cyclops.query.mimic`.

 - Handling conditions: `equals`, `in_`, `has_string_format`, `has_substring`, `starts_with`, `ends_with`
 - Attribute handling: `get_attributes`, `rename_attributes`, `reorder_attributes`, `drop_attributes`, `apply_to_attributes`
 - It automatically handles conversions between different query type objects, so users need not worry about what to pass in

## Conditions

Row conditions are represented by the `sqlalchemy.sql.elements.BinaryExpression` object.

In [ ]:
cond = db.mimic_hosp.diagnoses_icd.icd_code == "0011"
type(cond)

We can use method `where` to use a binary expression object to filter rows of a `sqlalchemy.sql.selectable.Select` object:

In [ ]:
cond = db.mimic_hosp.d_icd_diagnoses.icd_code == "0010"
query = select(db.mimic_hosp.d_icd_diagnoses.data).where(cond)
db.run_query(query)

Method `filter` may be used equivalently:

In [ ]:
cond = db.mimic_hosp.d_icd_diagnoses.icd_code == "0010"
query = select(db.mimic_hosp.d_icd_diagnoses.data).filter(cond)
db.run_query(query)

More complex conditions have been handled in `cyclops.query_utils`, available for convenience.

Consider the substring condition:

In [ ]:
cond = q_utils.substring_cond(db.mimic_hosp.d_icd_diagnoses.long_title, "Heart-")
query = select(db.mimic_hosp.d_icd_diagnoses.data).where(cond)
db.run_query(query)

Notice that we received results with "heart-" despite specifying "Heart-". This is because the function automatically assumed we wanted to ignore case. Different pre-processing assumptions are made for condition functions and can be explored in `cyclops.query_utils`.

These pre-processing assumptions are entirely optional. If case matters, we could simply specify this:

In [ ]:
cond = q_utils.substring_cond(
    db.mimic_hosp.d_icd_diagnoses.long_title, "Heart-", lower=False
)
query = select(db.mimic_hosp.d_icd_diagnoses.data).where(cond)
db.run_query(query)

Assumptions may be made regarding case, trimming of whitespace, and type conversions.

## Joins

We can perform joins in SQLAlchemy.

For example, say we want to get a table back with patient diagnoses (found in `db.mimic_hosp.diagnoses_icd`), but include the titles of the diagnoses (found in `db.mimic_hosp.d_icd_diagnoses`).

When creating complex queries, many joins consist of at least one, if not two, subqueries. So, we'll demonstrate an inner join using a subquery:

In [ ]:
query = select(db.mimic_hosp.diagnoses_icd.data, subquery).join(
    subquery, db.mimic_hosp.diagnoses_icd.icd_code == subquery.c.icd_code
)
db.run_query(query, limit=3)

A cross product can be done by simply not using the `join` method:

In [ ]:
subquery = select(db.mimic_hosp.d_icd_diagnoses.data).subquery()
query = select(db.mimic_hosp.diagnoses_icd.data, subquery)
db.run_query(query, limit=3)

It is smart to test queries with joins using a small `limit` in `db.run_query`, since cross products can create unreasonably large tables.

Notice that in the inner join we have duplicated, unwanted columns `icd_code_1`, `icd_version_1`. This is because we selected the entire `db.mimic_hosp.d_icd_diagnoses` table despite only wanting the `long_title` column.

We can fix this by selecting this column specifically:

In [ ]:
subquery = select(db.mimic_hosp.d_icd_diagnoses.data).subquery()

running_query = select(db.mimic_hosp.diagnoses_icd.data, subquery.c.long_title).join(
    subquery, db.mimic_hosp.diagnoses_icd.icd_code == subquery.c.icd_code
)
db.run_query(running_query, limit=3)

We can select any combination of tables and columns in this manner.

## Group by, Order by

We can perform use the `group_by` method to group by different columns:

In [ ]:
query = select(
    db.mimic_hosp.diagnoses_icd.icd_code,
    func.count(db.mimic_hosp.diagnoses_icd.icd_code),
).group_by(db.mimic_hosp.diagnoses_icd.icd_code)
db.run_query(query, limit=3)

The typically restrictions apply that anything selected must be in an aggregate function or included in the group by.

We can also take the previous query and order the rows in ascending or descending order:

In [ ]:
# Descending
query = query.order_by(db.mimic_hosp.diagnoses_icd.icd_code.desc())
db.run_query(query, limit=3)

In [ ]:
# Back to ascending
subquery = query.subquery()
query = select(subquery).order_by(subquery.c.icd_code)
db.run_query(query, limit=3)

## Helper functions

Let's re-order the attributes in previously defined query, perhaps where we want `icd_code` and its corresponding title, `long_title`, next to one another:

In [ ]:
running_query = q_utils.reorder_attributes(
    running_query,
    ["subject_id", "hadm_id", "seq_num", "icd_code", "long_title", "icd_version"],
)

db.run_query(running_query, limit=2)

Note that all existing attributes should be passed into this function, otherwise we'll receive an error.

We can also drop any attributes we might want removed:

In [ ]:
query_drop = q_utils.drop_attributes(running_query, "seq_num")
db.run_query(query_drop, limit=2)

Like many of the `q_utils` functions which accept attributes, we may pass in a single attribute, or multiple:

In [ ]:
query_drop = q_utils.drop_attributes(running_query, ["seq_num", "long_title"])
db.run_query(query_drop, limit=2)

We can rename attributes:

In [ ]:
query_rename = q_utils.rename_attributes(running_query, {"long_title": "icd_title"})
db.run_query(query_rename, limit=2)

### Applying functions

We need to be careful. Consider the diagnosis ICD code values:

In [ ]:
query = db.mimic_hosp.d_icd_diagnoses
df = db.run_query(query, limit=2)
df

In [ ]:
df["icd_code"].values

There is lots of trailing whitespace which we may not have expected.

There are built-in function in SQLAlchemy such as sqlalchemy.func.trim which can remove leading/trailing whitespace, however it is difficult to apply these function in-place. We have created functionality for this.

Here, we use `q_utils.trim_attributes` to trim leading/trailing whitespace from an attribute:

In [ ]:
query = q_utils.trim_attributes(db.mimic_hosp.d_icd_diagnoses, "icd_code")
df = db.run_query(query, limit=2)
df["icd_code"].values

It is smart to perform sanity checks:

In [ ]:
t = db.mimic_hosp.d_icd_diagnoses
df = db.run_query(t, limit=5)
icd_codes_untrimmed = df["icd_code"].values.astype("str")
np.char.strip(icd_codes_untrimmed) == icd_codes_untrimmed  # Trim using NumPy

In [ ]:
query = q_utils.trim_attributes(t, "icd_code")
df = db.run_query(query, limit=5)
icd_codes_trimmed = df["icd_code"].values.astype("str")
np.char.strip(icd_codes_trimmed) == icd_codes_trimmed  # Trim using NumPy

Like many of the functions which accept attributes, we may pass in a single attribute, or multiple:

In [ ]:
query = q_utils.trim_attributes(
    db.mimic_hosp.d_icd_diagnoses, ["icd_code", "icd_version"]
)
df = db.run_query(query, limit=2)
df[["icd_code", "icd_version"]].values

Note that certain functions will assume certain datatype conversions. For example, when trimming, the column would be converted to a string in order for this operation to be defined.

The ICD version was originally not a string:

In [ ]:
query = db.mimic_hosp.d_icd_diagnoses
df = db.run_query(query, limit=2)
df["icd_version"].values

String conversions are made with `string_format_cond`, `substring_cond`, `startswith_cond`, `endswith_cond`. For example,

In [ ]:
query = select(db.mimic_hosp.diagnoses_icd.data).where(
    q_utils.startswith_cond(db.mimic_hosp.diagnoses_icd.icd_version, 1)
)
db.run_query(query, limit=3)

There is also support for creating custom in-place functions using `q_utils.apply_to_attributes`.

Here we will use `sqlalchemy.func.lower` to convert the strings in a column to lowercase:

In [ ]:
query = q_utils.apply_to_attributes(
    db.mimic_hosp.d_icd_diagnoses, "long_title", func.lower
)
db.run_query(query, limit=2)

In particular, `q_utils.apply_to_attributes` accepts any function which takes a Column object as its only argument and similarly returns a Column object.